In [1]:
!pip install opencv-python
import os
import cv2
import numpy as np
import pandas as pd
from IPython.display import HTML
from base64 import b64encode

In [2]:
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy

import itertools
import math

import mediapy as media

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.2 MB/s eta 0:00:00


In [21]:
df = pd.DataFrame({"Video_name":['video'], "Frames_to_cut":[[20, 100]]})

In [22]:
df

,Video_name,Frames_to_cut
0,video,"[20, 100]"


In [25]:
class MovieFragmentCutter():

  def __init__(self, save_folder_path, display = True):
    self.save_folder_path = save_folder_path
    self.display = display

  def create_new_video(self, video_name, frames, fps, H, W):
    video_path = '{}/{}_fr{}to{}.avi'.format(self.save_folder_path, video_name, frames[0], frames[1])
    video_writer = cv2.VideoWriter_fourcc('M','J','P','G')
    video_file = cv2.VideoWriter(video_path, video_writer, fps, (W, H), True)
    return video_file, video_path

  def show_video(self, video_path, fps):
    video2 = media.read_video(video_path)
    video2 = media.resize_video(video2, np.array(video2.shape[1:3]) // 2)
    media.show_video(video2, fps=30)

  def get_frames_from_video(self, capture, frames, video_name):
    first_frame = int(frames[0])
    last_frame = int(frames[1])

    fps = int(capture.get(cv2.CAP_PROP_FPS))
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < last_frame:
      print('Movie is too short. It\'\s has {} frames, and you give last frame {}'. format(frame_count, last_frame))
      return

    frame = 0
    images_list = []

    while True:
      success, image = capture.read()
      if image is None or success is None:
        break

      H, W = image.shape[:2]

      if first_frame == frame:
        video_file, video_path = self.create_new_video(video_name, frames, fps, H, W)
      elif first_frame < frame and frame < last_frame:
        video_file.write(image)
        cv2.waitKey(int(1000/fps))
      elif frame == last_frame:
        video_file.write(image)

        capture.release()
        video_file.release()
        cv2.destroyAllWindows()
        break

      frame = frame +1

    return video_path, fps

  def get_specific_video_frames(self, df):
    for i, row in df[['Video_name', 'Frames_to_cut']].iterrows():
      video_name = row['Video_name']
      print(video_name)
      input_path= (f'{self.save_folder_path}{video_name}.mp4')
      print(input_path)
      frames = row['Frames_to_cut']
      capture = cv2.VideoCapture(input_path)
      video_path, fps = self.get_frames_from_video(capture, frames, video_name)

      if self.display:
        self.show_video(video_path, fps)

In [26]:
save_folder_path = '/content/drive/MyDrive/'

movie_cutter = MovieFragmentCutter(save_folder_path, display = True)

movie_cutter.get_specific_video_frames(df)

video
/content/drive/MyDrive/PASCAL/video.mp4


This browser does not support the video tag.
